## Random Walk 解析

这里，我们理解一下DGL里关于在异构图上进行随机游走的API，观察随机游走产生的metapath的情况。

<img src='heterogeneous_graph.png' width=40%>

In [10]:
import dgl
from dgl.sampling import random_walk

#### 导入保存的异构图

In [3]:
graphs, _ = dgl.load_graphs('./hetero_dgl.bin')
graph = graphs[0]

In [5]:
print(graph)

Graph(num_nodes={'company': 3, 'product': 8},
      num_edges={('company', 'produce', 'product'): 8, ('company', 'suppliedby', 'company'): 3, ('company', 'supply', 'company'): 3, ('product', 'downstream', 'product'): 7, ('product', 'producedby', 'company'): 8, ('product', 'upstream', 'product'): 7},
      metagraph=[('company', 'product', 'produce'), ('company', 'company', 'suppliedby'), ('company', 'company', 'supply'), ('product', 'product', 'downstream'), ('product', 'product', 'upstream'), ('product', 'company', 'producedby')])


### 设定metapath的schema

在异构图里，随机游走一般会遵循一些有业务意义的路径(metapath)。定义这些metapath需要结合实际业务和需求来确定。这种metapath的类型叫schema。在我们这个样例图里面，从公司的角度出发，可以定义一种metapath的schema，即：公司-〈生产〉-产品-〈下游〉-产品-〈被生产〉-公司。我们这里使用这个schema来进行随机游走。

对于游走的出发点，我们选择公司`comp_2`。它的点ID是1。我们对它进行2次随机游走。

In [6]:
trace,_ = random_walk(graph, [1] * 7, metapath=['produce', 'downstream', 'producedby'])

In [7]:
trace

tensor([[1, 4, 1, 0],
        [1, 3, 4, 1],
        [1, 5, 4, 1],
        [1, 4, 6, 2],
        [1, 4, 1, 0],
        [1, 4, 6, 2],
        [1, 3, 4, 1]])

对于相同的metapath，我们还可以进行更多跳的游走。比如下面的例子里，我们就让 公司-〈生产〉-产品-〈下游〉-产品-〈被生产〉-公司 这个路径再多走一遍。实现也很简单，就是在metapath那里再复制一遍路径即可。

In [8]:
trace2,_ = random_walk(graph, [1] * 7, metapath=['produce', 'downstream', 'producedby']*2)

In [9]:
trace2

tensor([[ 1,  5,  4,  1,  4,  1,  0],
        [ 1,  3,  4,  1,  5,  4,  1],
        [ 1,  4,  6,  2,  7, -1, -1],
        [ 1,  4,  1,  0,  2,  3,  1],
        [ 1,  3,  4,  1,  5,  4,  1],
        [ 1,  3,  4,  1,  4,  1,  0],
        [ 1,  5,  4,  1,  5,  4,  1]])